In [1]:
from automatic_circuits.mamba import Mamba, ModelArgs

In [2]:
args = ModelArgs(
    d_model = 64,
    n_layer = 2,
    vocab_size = 3,
    d_state = 64,
    expand = 2,
    dt_rank = 'auto',
    d_conv = 2,
    pad_vocab_size_multiple = 8,
    conv_bias = True,
    bias = False
)

In [3]:
model = Mamba(args)

In [4]:
def generate_cum_parity(total_seq_len: int, rng):
    sep = np.array([2])
    prob = rng.beta(2, 2)
    bits = rng.choice(2, (total_seq_len - 1,), replace=True, p=[prob, 1. - prob])
    parities = np.concatenate([sep, np.cumsum(bits) % 2])
    bits = np.concatenate([sep, bits])
    return bits, parities


def generate_fixed_parity(seq_length=64, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    sep = 2
    #sequence_lengths = [seq_length] * (total_seq_length // seq_length)
    #assert sum(sequence_lengths) <= total_seq_length
    bits, parities = generate_cum_parity(seq_length, rng)
    
    bits = np.concatenate(bits)
    parities = np.concatenate(parities)
    diff = total_seq_length - len(parities)
    bits = np.pad(bits, (0, diff), mode='constant', constant_values=(sep, sep))
    parities = np.pad(parities, (0, diff), mode='constant', constant_values=(sep, sep))
    return bits, parities

In [5]:
import numpy as np
import torch

rng = np.random.default_rng(0)

bits, parities = generate_cum_parity(128, rng)

bits = torch.asarray(bits)
parities = torch.asarray(parities)

In [15]:
def generate_cum_parity(seq_len: int, batch_size: int):
    dist = torch.distributions.beta.Beta(2., 2.)
    probs = dist.sample((batch_size, 1))
    bits = torch.bernoulli(torch.ones((batch_size, seq_len)) * probs)
    parities = bits.cumsum(dim=1) % 2
    return bits, parities

In [17]:
bits, parities = generate_cum_parity(64, 10)
print(bits.shape)

torch.Size([10, 64])
